<a href="https://colab.research.google.com/github/miguelamda/TL-tutorial/blob/master/TransferLearning-Ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1. Transfer Learning con VGG16

Ejercicio 1 del tutorial de Transfer Learning.

GPT2: Diseño y Gestión de Proyectos en Data Science II.
[Máster en Data Science y Big Data](http://masterds.es/) de la [Universidad de Sevilla](http://www.us.es). 

25/06/2020. Profesor: [Miguel Ángel Martínez del Amor](http://www.cs.us.es/~mdelamor)

Este ejercicio puede ayudar a mejorar las habilidades con Keras. Para ello se propone emplear el modelo VGG16, el cual es más sencillo y que ya habéis visto en clase (o accediendo [aquí](https://github.com/fsancho/DL/blob/master/4.%20Redes%20Convolucionales/4.3.%20CNN%20Preentrenadas.ipynb)). 

## 1. Importación de librerías y funciones auxiliares <a class="anchor" id="transferimp"></a>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import numpy as np
import os

A continuación la importación de la API de Keras. Como **primer ejercicio**, busca como se llama la función que carga el modelo VGG16, e importala en la celda siguiente.

In [ ]:
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop

# Importa a continuación la función que carga el modelo VGG16
#from keras.applications import ???
from keras.applications.vgg16 import preprocess_input, decode_predictions

tf.__version__

Vamos a usar las mismas funciones auxiliares que en el tutorial, las tienes a continuación, todas en una sola celda para definirlas más rápido.

In [ ]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

def plot_images(images, cls_true, cls_pred=None, smooth=True):

    assert len(images) == len(cls_true)

    # Crea una figura con sub-gráficas.
    fig, axes = plt.subplots(3, 3)

    # Ajusta el espacio vertical.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    # Tipo de interpolación.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'

    for i, ax in enumerate(axes.flat):
        # Puede haber menos de 9 imágenes, nos aseguramos que no falle.
        if i < len(images):
            # Dibuja imagen.
            ax.imshow(images[i],
                      interpolation=interpolation)

            # Number de la true class.
            cls_true_name = class_names[cls_true[i]]

            # Muestra clases predichas y verdaderas.
            if cls_pred is None:
                xlabel = "True: {0}".format(cls_true_name)
            else:
                # Nombre de la clase predicha.
                cls_pred_name = class_names[cls_pred[i]]

                xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

            # Muestra las clases con la etiqueta en el eje x.
            ax.set_xlabel(xlabel)
        
        # Elimina ticks en la gráfica.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Asegurar que la gráfica se muestra correctamente con gráficos múltiples
    # en una sola celda Notebook.
    plt.show()
    
# Importa una función de sklearn para calcular la matriz de confusión.
from sklearn.metrics import confusion_matrix

def print_confusion_matrix(cls_pred):
    # cls_pred es un array del número de la clase predicha para
    # todas las imágenes del conjunto de test.

    # Obtiene la matriz de confusión usando sklearn.
    cm = confusion_matrix(y_true=cls_test,  # True class para el conjunto de test.
                          y_pred=cls_pred)  # Predicted class.

    print("Matriz de confusión:")
    
    # Imprime la matriz de confusión como texto.
    print(cm)
    
    # Imprime los nombres de clases para facilitar la referencia.
    for i, class_name in enumerate(class_names):
        print("({0}) {1}".format(i, class_name))
        
def plot_example_errors(cls_pred):
    # cls_pred es un array del número de la clase predicha para
    # todas las imágenes en el conjunto de test.

    # Array booleano indicando si la clase predicha es incorrecta.
    incorrect = (cls_pred != cls_test)

    # Obtiene las rutas de ficheros para las imágenes que son clasificadas incorrectamente.
    image_paths = np.array(image_paths_test)[incorrect]

    # Carga las primeras 9 imágenes.
    images = load_images(image_paths=image_paths[0:9])
    
    # Obtiene las clases predichas para esas imágenes.
    cls_pred = cls_pred[incorrect]

    # Obtiene las clases de verdad para esas imágenes.
    cls_true = cls_test[incorrect]
    
    # Muestra las 9 imágenes que hemos cargado y sus correspondientes clases.
    # Tenemos solo 9 imágenes, por lo que no hace falta dividirlas otra vez.
    plot_images(images=images,
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])
    
def example_errors(model=None):
    # El generador de datos de Keras para el conjunto de test se debe resetear
    # antes del procesamiento. Esto es porque el generador va a iterar
    # infintamente y mantendrá un índice interno en el dataset.
    # Por tanto, se podrá comenzar por el medio del conjunto de test si no lo
    # reseteamos primero. Esto imposibilita encajar las clases predichas con
    # las imágenes de entrada. Si reseteamos el generador, entonces siempre
    # compienza por el comienzo, así que sabemos exáctamente qué imágenes
    # de entrada se están usando.
    if model is None:
        model = new_model
        
    generator_test.reset()
    
    # Predecir las clases para todas las imágenes del conjunto de test.
    y_pred = model.predict_generator(generator_test,
                                      steps=steps_test)

    # Convertir las clases predichas de arrays a enteros.
    cls_pred = np.argmax(y_pred,axis=1)

    # Muestra los ejemplos de imágenes mal clasificados.
    plot_example_errors(cls_pred)
    
    # Muestra la matriz de confusión.
    print_confusion_matrix(cls_pred)
    
def load_images(image_paths):
    # Carga las imágenes de disco.
    images = [plt.imread(path) for path in image_paths]

    # Convierte a un array de numpy y lo devuelve.
    return np.asarray(images)

def plot_training_history(history):
    # Obtiene la precisión de clasificación y el valor de pérdida para el
    # conjunto de entrenamiento.
    acc = history.history['categorical_accuracy']
    loss = history.history['loss']

    # También para el conjunto de validación (solo usamos el del conjunto de test).
    val_acc = history.history['val_categorical_accuracy']
    val_loss = history.history['val_loss']

    # Muestra el valor del accuracy y pérdida para el conjunto de entrenamiento.
    plt.plot(acc, linestyle='-', color='b', label='Training Acc.')
    plt.plot(loss, 'o', color='b', label='Training Loss')
    
    # Muestra el del conjunto de test.
    plt.plot(val_acc, linestyle='--', color='r', label='Test Acc.')
    plt.plot(val_loss, 'o', color='r', label='Test Loss')

    # Muestra el título y la leyenda.
    plt.title('Training and Test Accuracy')
    plt.legend()

    # Se asegura de mostrar la gráfica correctamente.
    plt.show()

## 2. El Modelo Pre-Entrenado: VGG16 

Lo siguiente crea una instancia del modelo VGG16 pre-entrenado usando la API de [Keras](https://keras.io/). Esto descarga automáticamente los archivos necesarios si no los tiene ya. 

El modelo VGG16 contiene una parte convolucional y una parte completamente conectada (o densa) que se utiliza para la clasificación. Si `include_top=True` entonces se descarga todo el modelo VGG16 que tiene unos 528 MB. Si `include_top=False` entonces sólo se descarga la parte convolucional del modelo VGG16, que es de sólo 57 MB. Descaragaremos esta última versión.

![VGG model](https://github.com/miguelamda/TL-tutorial/blob/master/images/11_vgg_model.png?raw=1)

In [ ]:
vggmodel = ????

## 3. El Dataset: Knifey-Spoony 

Carga el dataset tal y como se vió en el tutorial. A continuación las líneas de código.

In [ ]:
# Carga el dataset empleando el fichero kinfey.py
import knifey

# Descarga el dataset, si no se ha descargado ya
knifey.maybe_download_and_extract()

# Adapta la estructura de carpetas para Keras
knifey.copy_files()

# Define las rutas a los directorios de train y test
train_dir = knifey.train_dir
test_dir = knifey.test_dir

## 4. El Canal de Entrada

Para definir el pipeline de entrada para el modelo, primero necesitamos saber la forma de los tensores esperados como entrada por el modelo VGG16 pre-entrenado. En este caso, ¿qué forma tienen las imágenes de entrada?

In [ ]:
input_shape = ????

Define a continuación un *generador de datos* que haga aumentado mediante transformaciones aleatorias. Para VGG16, es necesario tan solo reescalar los píxeles a 1.0/255, así que no hace falta usar la función de preprocesamiento de entrada.

In [ ]:
datagen_train = ????
datagen_test = ????

Debido a que el modelo VGG16 es muy grande, el tamaño del batch no puede ser demasiado grande.

In [ ]:
batch_size = 20

Podemos guardar las imágenes transformadas aleatoriamente durante el entrenamiento, para comprobar si han sido demasiado distorsionadas, por lo que tendríamos que ajustar los parámetros del generador de datos anterior.

In [ ]:
if True:
    save_to_dir = None
else:
    save_to_dir='augmented_images/'

Ahora creamos el generador de datos real que leerá los archivos del disco, redimensionará las imágenes y devolverá un lote aleatorio.

In [ ]:
generator_train = ????

generator_test = ????

Debido a que nuestro conjunto de pruebas contiene 530 imágenes y el tamaño del batch está configurado en 20, el número de pasos es 26,5 para un procesamiento completo del conjunto de pruebas.

In [ ]:
steps_test = generator_test.n / batch_size
steps_test

## 5. Clases del Conjunto de Datos 

Obtengamos las rutas de los ficheros para todas las imágenes en los conjuntos de entrenamiento y de pruebas.

In [ ]:
image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)

Obtengamos también las clasificaciones reales (el número correspondiente) de cada imagen en los conjuntos de training y test.

In [ ]:
cls_train = generator_train.classes
cls_test = generator_test.classes

Obtengamos los nombres correspondientes de las clases del dataset y el número de ellos.

In [ ]:
class_names = list(generator_train.class_indices.keys())
num_classes = generator_train.num_classes

Dado que el conjunto de datos Knifey-Spoony está bastante desequilibrado porque tiene pocas imágenes de tenedores, más imágenes de cuchillos y muchas más imágenes de cucharas. Así que vamos a calcular **pesos** que equilibrarán adecuadamente el conjunto de datos.

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)

## 6. Transfer Learning

Primero imprimimos un resumen del modelo VGG16 para poder ver los nombres y tipos de sus capas, así como las formas de los tensores que fluyen entre las capas.

In [ ]:
vggmodel.????

En este ejercicio vamos a extraer la parte convolucional de forma personalizada, es decir, desde la entrada hasta una capa deseada (de esta forma podrás hacer transfer learning desde otras capas). Comprueba como difiere esto a como lo habéis hecho en clase anteriormente (o [aquí](https://github.com/fsancho/DL/blob/master/4.%20Redes%20Convolucionales/4.3.%20CNN%20Preentrenadas.ipynb)).

Podemos ver que la última capa convolucional se llama 'block5_pool', y podemos usar Keras para obtener una referencia a dicha capa.

In [ ]:
transfer_layer = vggmodel.get_layer('block5_pool')

Nos referiremos a esta capa como la Capa de Transferencia (**Transfer Layer**), puesto que su salida será re-enrutada a nuestra nueva red neuronal completamente conectada que hará la clasificación final sobre el Knifey-Spoony dataset.

La salida de la capa de transferencia tiene la siguiente forma:

In [ ]:
transfer_layer.output

Es muy sencillo crear un nuevo modelo usando la API de Keras. Primero tomamos la parte del modelo VGG16 desde su capa de entrada hasta la salida de la capa de transferencia. Podemos llamarlo el modelo convolucional, porque consiste en todas las capas convolucionales del modelo VGG16.

In [ ]:
conv_model = Model(inputs=vggmodel.input,
                   outputs=transfer_layer.output)

Podemos entonces usar Keras para construir un modelo nuevo encima de este.

In [ ]:
# Creamos un nuevo modelo Secuencial de Keras
nuevo_modelo = ????

# Añadimos la parte convolucional del modelo VGG16 de arriba
nuevo_modelo.add(????)

# Aplanamos la salida del modelo VGG16 dado que ésta viene
# de una capa convolucional.
nuevo_modelo.add(????)

# Añade una capa densa (es decir, totalmente conectada o fully-connected).
# Esto es para combinar las características que el modelo VGG16 ha
# reconocido en la imagen. Usa como función de activación ReLu.
nuevo_modelo.add(????)

# Añade una capa dropout el cual prevendrá el sobreajuste y mejorará
# la capacidad de generalización en datos desconocidos (es decir, el 
# conjunto de test). Usa un ratio de 0.5
nuevo_modelo.add(????)

# Añade la capa final para la clasificación real, usando softmax.
nuevo_modelo.add(????)

Utilizamos el optimizador Adam con una tasa de aprendizaje bastante baja de 1e-5. La tasa de aprendizaje podría ser mayor, pero si se intenta entrenar más capas del modelo original VGG16, entonces la velocidad de aprendizaje debería ser bastante baja, de lo contrario los pesos preentrenados del modelo VGG16 se distorsionarán y no podrá aprender.

In [ ]:
optimizer = ????

Tenemos 3 clases en el Knifey-Spoony dataset, por lo que Keras necesita usar una **función de pérdida** (loss function).

In [ ]:
loss = ????

La única **métrica de rendimiento** en la que estamos interesados es en la precisión de clasificación (clasiffication accuracy).

In [ ]:
metrics = ????

Función auxiliar para imprimir si la capa en el modelo VGG16 debe ser entrenada.

In [ ]:
def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))
        
example_errors()

Por defecto, todas las capas del modelo VGG16 son entrenables.

En transfer learning estamos inicialmente interesados tan solo en reusar el modelo VGG16 tal cual, como un **extractor de características**, por lo que deshabilitaremos el entrenamiento en todas sus capas.

In [ ]:
conv_model.???? = False

In [ ]:
for layer in ????:
    layer.???? = False

In [ ]:
print_layer_trainable()

Una vez que hayamos cambiado si las capas del modelos son entrenables, necesitamos compilarlo para que los cambios surtan efecto.

In [ ]:
nuevo_modelo.????

A continuación entrenamos el nuevo modelo, lo que se hace con tan solo una llamada a una función en la API de Keras. Definimos 20 épocas y 100 pasos por época (ya que tenemos batches de 20).

In [ ]:
epochs = 20
steps_per_epoch = 100
history = nuevo_modelo.????

Mostremos la gráfica de evolución de las métricas.

In [ ]:
plot_training_history(history)

Después del entrenamiento también podemos evaluar el rendimiento del nuevo modelo en el conjunto de pruebas usando una sola llamada de función en la API de Keras.

In [ ]:
result = nuevo_modelo.????

In [ ]:
print("Test-set classification accuracy: {0:.2%}".format(result[1]))

Podemos representar algunos ejemplos de imágenes mal clasificadas del conjunto de pruebas. 

In [ ]:
example_errors(model=nuevo_modelo)

### 7. Fine-Tuning

Podemos tratar de afinar suavemente algunas de las capas más profundas del modelo VGG16 también. A esto lo llamamos "Ajuste fino", o **Fine Tuning**.

No está claro si Keras usa el booleano `trainable` en cada capa del modelo original VGG16 o si es anulado por el booleano `trainable` en la'meta-capa' que llamamos `conv_layer`. Así que habilitaremos el booleano `trainable` tanto para `conv_layer` como para todas las capas relevantes en el modelo original VGG16.

In [ ]:
conv_model.???? = True

Queremos entrenar las últimas dos capas convolucionales, es decir, cuyos nombres contienen 'block5' o 'block4'.

In [ ]:
for layer in conv_model.????:
    # Booleano de si la capa es entrenable
    trainable = ('block5' in layer.name or 'block4' in layer.name)
    
    # Ajusta el booleano de la capa
    layer.???? = trainable

Podemos comprobar que esto ha actualizado el booleano `trainable` para las capas relevantes.

In [ ]:
print_layer_trainable()

Usaremos el optimizador Adam con un bajo factor de aprendizaje bajo para el ajuste fino, 1e-7.

In [ ]:
optimizer_fine = ????

Dado que hemos definido un nuevo optimizador y hemos cambiado los booleanos `trainable` para muchas de las capas en el modelo, necesitamos recompilarlo para que los cambios hagan efecto.

In [ ]:
nuevo_modelo.????

Continuamos por tanto con el entrenamiento por donde lo dejamos anteriormente, ahora aplicando fine-tuning al modelo VGG16 y el nuevo clasificador. Sigamos con 20 épocas.

In [ ]:
history = nuevo_modelo.????

Luego podemos mostrar gráficamente los valores de pérdida y precisión de la clasificación a partir del entrenamiento. 

In [ ]:
plot_training_history(history)

In [ ]:
result = nuevo_modelo.????

In [ ]:
print("Test-set classification accuracy: {0:.2%}".format(result[1]))

Podemos volver a mostrar algunos ejemplos de imágenes mal clasificadas, y también podemos ver en la matriz de confusión que el modelo sigue teniendo problemas para clasificar correctamente los tenedores.

In [ ]:
example_errors(model=nuevo_modelo)

## License (MIT)

Based on the TensorFlow tutorials by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

Copyright (c) 2016-2017 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.